In [ ]:
print("Let's start!\n")
print("Loading...")

import glob
import random
print("...")


import tensorflow as tf
import pandas as pd
print("...")

import librosa
import librosa.display
print("...")

import json
print("...")
import numpy as np
from sklearn import metrics
print("...")


print("Libraries are imported successfully")

#!pip install --target="/content/drive/My Drive/Colab_PackagesandLibraries" wget
#import wget

# Install files to spesific directory
#!wget "https://ndownloader.figshare.com/files/10853306"
#output_directory = "/content/drive/My Drive/Datasets/warblrb10k/warblrb10k_public_metadata_2018.csv"
#wget.download(url, out=output_directory)

#url= "https://ndownloader.figshare.com/files/10853303" 
#output_directory = "/content/drive/My Drive/Datasets/ff1010bird/ff1010bird_metadata_2018.csv"
#wget.download(url, out=output_directory)

# load data
#!wget https://archive.org/download/ff1010bird/ff1010bird_wav.zip

#!wget https://archive.org/download/warblrb10k_public/warblrb10k_public_wav.zip

# Unzip folder to spesific directory
#!unzip ff1010bird_wav.zip -d "/content/drive/My Drive/Datasets/ff1010bird"

#!unzip warblrb10k_public_wav.zip -d "/content/drive/My Drive/Datasets/warblrb10k"

Let's start!

Loading...
...
...
...
...
...
Libraries are imported successfully


In [ ]:
# Getting the dataset
# ff1010bird, warblrb10k
datafolder = "/content/drive/My Drive/Datasets/ff1010bird/"
#datafolder = "/content/drive/My Drive/Datasets/warblrb10k/"
loc = datafolder+"wav/"
metadata = pd.read_csv(datafolder+"metadata.csv")
metadataFiles = [loc+str(s)+".wav" for s in metadata["itemid"]]
#print(metadataFiles)
metadataLabels = [str(s) for s in metadata["hasbird"]]
#print(metadataLabels)
metadataFiles = np.vstack((metadataFiles, metadataLabels)).T
#print(metadataFiles)

In [ ]:
# Function for extracting features
def extract_features(dir,
                     bands=40,frames=41):
  def _windows(data, window_size):
    start = 0
    while start < len(data):
      yield int(start), int(start + window_size)
      start += (window_size // 2)


  window_size = 512 * (frames - 1)
  features, labels = [], []
  for fn in dir:
    segment_log_specgrams, segment_labels = [], []
    sound_clip,sr = librosa.load(fn[0])
    label = fn[1]
    for (start,end) in _windows(sound_clip,window_size):
      if(len(sound_clip[start:end]) == window_size):
        signal = sound_clip[start:end]
        melspec = librosa.feature.melspectrogram(signal,n_mels=bands)
        logspec = librosa.amplitude_to_db(melspec)
        logspec = logspec.T.flatten()[:, np.newaxis].T
        segment_log_specgrams.append(logspec)
        segment_labels.append(label)

    segment_log_specgrams = np.asarray(segment_log_specgrams).reshape(
                len(segment_log_specgrams),bands,frames,1)
    segment_features = np.concatenate((segment_log_specgrams, np.zeros(
                np.shape(segment_log_specgrams))), axis=3)
    for i in range(len(segment_features)):
      segment_features[i, :, :, 1] = librosa.feature.delta(
                    segment_features[i, :, :, 0])
    if True:#len(segment_features) > 0: # check for empty segments
      if len(labels)==0:
        features = segment_features
        labels = segment_labels
      else:
        features = np.concatenate((features,segment_features))
        labels =np.concatenate((labels,segment_labels))
  return features, labels
print("What the heck")
# Extracting features from sound files and save them to file
"""for i in range (int (len(metadataFiles)/100)+1):
  if (len(metadataFiles)%100)==0 and i==(len(metadataFiles)/100):
    break
  if i==(len(metadataFiles)/100):
    featureVectors, VectorLabels=extract_features(metadataFiles[:][i*100:len(metadataFiles)])
  else:
    featureVectors, VectorLabels=extract_features(metadataFiles[:][i*100:(i+1)*100])
  featureVectors = np.array(featureVectors)
  VectorLabels= np.array(VectorLabels)
  with open(datafolder+'mfcc_mfccdelta/'+'features_'+str(i)+'.npy', 'wb') as f:
    np.save(f,featureVectors)
  with open(datafolder+'mfcc_mfccdelta/'+'labels_'+str(i)+'.npy', 'wb') as f:
    np.save(f, VectorLabels)
  print(str(i))"""

del metadataFiles, metadataLabels

What the heck
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76


In [ ]:
#Load features and labels from trainFiles
for i in range (int (len(metadata)/100)+1):
  if (len(metadata)%100)==0 and i==(len(metadata)/100):
    break
  with open(datafolder+'mfcc_mfccdelta/'+'features_'+str(i)+'.npy','rb') as fs:
    featureVectorstmp = np.load(fs)
  with open(datafolder+'mfcc_mfccdelta/'+'labels_'+str(i)+'.npy','rb') as f:
    VectorLabelstmp = np.load(f)
  if i==0:
    featureVectors=featureVectorstmp
    VectorLabels=VectorLabelstmp
  else:
    featureVectors=np.concatenate((featureVectors,featureVectorstmp))
    VectorLabels=np.concatenate((VectorLabels,VectorLabelstmp))
  fs.close()
  f.close()
print(featureVectors.shape)
print("Features and labels loaded")


(153800, 40, 41, 2)
Features and labels loaded


In [ ]:
#Determine train and dev sets from the dataset
print(len(featureVectors))
m = len(VectorLabels)
P = 0.9
idx = np.random.permutation(m)
trainfeatureVectors=featureVectors[idx[:round(P*m)]]
trainVectorLabels=VectorLabels[idx[:round(P*m)]]
ValidationfeatureVectors=featureVectors[idx[round(P*m):round(m)]]
ValidationVectorLabels=VectorLabels[idx[round(P*m):round(m)]]
del featureVectors, VectorLabels

153800


In [ ]:
# Reshaping (flattening) the extracted features to make them suitable for inserting the model
trainVectorLabels = np.array(trainVectorLabels).flatten()
trainVectorLabels[:] = np.array(trainVectorLabels[:])
trainVectorLabels = trainVectorLabels.reshape(len(trainVectorLabels),1)
ValidationVectorLabels = np.array(ValidationVectorLabels).flatten()
ValidationVectorLabels = ValidationVectorLabels.reshape(len(ValidationVectorLabels),1)

trainfeatureVectors = np.array(trainfeatureVectors).flatten()
trainfeatureVectors = trainfeatureVectors.reshape(int(trainfeatureVectors.shape[0]/len(trainVectorLabels)),len(trainVectorLabels))

# Normalizing the vector values
trainfeatureVectors = trainfeatureVectors/np.linalg.norm(trainfeatureVectors) #########

print("Dim of feature vectors of training set: " + str(trainfeatureVectors.shape))

ValidationfeatureVectors = np.array(ValidationfeatureVectors).flatten()
ValidationfeatureVectors = ValidationfeatureVectors.reshape(int(ValidationfeatureVectors.shape[0]/len(ValidationVectorLabels)),len(ValidationVectorLabels))

# Normalizing the vector values
ValidationfeatureVectors = ValidationfeatureVectors/np.linalg.norm(ValidationfeatureVectors) #########

print("Dim of feature vectors of validation set: " + str(ValidationfeatureVectors.shape))


Dim of feature vectors of training set: (3280, 138420)
Dim of feature vectors of validation set: (3280, 15380)


In [ ]:
# Constructing the model
model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=(trainfeatureVectors.shape[0])),
        # tf.keras.layers.experimental.RandomFourierFeatures(
        #     output_dim=4096, scale=10.0, kernel_initializer="gaussian"
        # ),
        tf.keras.layers.Dense(512, activation= "relu"),
        tf.keras.layers.Dense(256, activation = "linear"),
        tf.keras.layers.Dense(256, activation = "relu"),
        tf.keras.layers.Dense(256, activation = "relu"),
        tf.keras.layers.Dense(64, activation = "linear"),
        tf.keras.layers.Dense(64, activation = "relu"),
        tf.keras.layers.Dense(64, activation = "relu"),
        tf.keras.layers.Dense(32, activation = "relu"),
        tf.keras.layers.Dense(16, activation = "relu"),
        tf.keras.layers.Dense(1, activation = "sigmoid"),
    ]
)

# Compiling the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[tf.keras.metrics.BinaryAccuracy()],
)

In [ ]:
print(trainVectorLabels[0])

['0']


In [ ]:
# Fitting the training data into the model
history = model.fit(trainfeatureVectors.T.astype(np.float),trainVectorLabels.astype(np.float),batch_size = 512,epochs=50,verbose=0,shuffle=True)

model.summary()

#Evaluating and making predictoin with validation set
valid_loss, valid_acc = model.evaluate(ValidationfeatureVectors.T.astype(np.float), ValidationVectorLabels.astype(np.float))
predictions = model.predict(ValidationfeatureVectors.T.astype(np.float))

# predictions = model.predict(ValidationfeatureVectors_flat.T.astype(np.float))

print("Validation loss: " + str(valid_loss))
print("Validation accuracy: " + str(valid_acc))

print(predictions[0])


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               1679872   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_5 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_6 (Dense)              (None, 64)                4

In [ ]:
# Binarization of prediction data
predictions = (predictions>=0.5)

print(predictions[0])

[False]
